# Introduction/Business Problem
In the capstone project for IBM certificate course, the audience will be road users who will benefit from information that will possibly help reduce traffic collision/accidents in the future. This will be acheived by analysing the impact of accident "severity" whether in terms of fatality or traffic. All records were provided by SPD and recorded by Traffic Records and provided to me by Coursera. This dataset covers a period of 2004 to present. It contains information such as severity code, address type, weather, road condition, speeding, etc.

# Data Understanding

There are 194,673 observations and 38 attributes in the dataset. The aim is to identify the severity of an accident and the possible factors that can lead to one. From the dataset, the SEVERITYCODE attribute will be used as the target variable (Y). Feature Engineering process will be applied to ascertain the attributes that impact the severity of an accident such as WEATHER, ROADCOND, LIGHTCOND, etc. Missing value will either be replaced or removed from the dataset as well.